In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('ItemsApp').getOrCreate()

In [5]:
df=spark.read.csv('d:\\item_details.csv',header=True,inferSchema=True)

In [6]:
df.show()

+-------+---------+---------+--------+-----+---+
|item id|item name|item cost|supplier|grade|qty|
+-------+---------+---------+--------+-----+---+
|      8|    chalk|    65.76|       X|    A|  2|
|     12|   Pencil|     null|       Y|    B|  3|
|     13|      Pen|     43.0|       X|    A|  5|
|     15|   Duster|     54.0|       Y|    C|  1|
|     23|    Scale|     10.0|    null|    A|  2|
|      8|    chalk|    65.76|       X|    A|  2|
+-------+---------+---------+--------+-----+---+



In [7]:
df.limit(3).toPandas()

,item id,item name,item cost,supplier,grade,qty
0,8,chalk,65.76,X,A,2
1,12,Pencil,NaN,Y,B,3
2,13,Pen,43.00,X,A,5


In [ ]:
From two separate CSV files (items.csv,Supplier.csv) fetch data in two pyspark dataframes.  
Please complete following tasks given with based on it....                                                                                             #1.show item details and Supplier details supplied by MR.X
#2.show supplier details who has supplied items with cost>1000
#3.show all item details whos supplier details are not available
#4.show item details whose supplier details are available
#5.show item details along with supplier details for such items ,for which supplier details are  available and item name starts with 'b'
#6.Show supplier wise number of items supplied, sum ,min ,max total of itemcost .
#all item cost supplied
#7.join overall items available in 2 stores

In [103]:
df=spark.read.csv('d:\\item.csv',header=True,inferSchema=True)

In [104]:
df1=spark.read.csv('d:\\supplier.csv',header=True,inferSchema=True)

In [105]:
df.createOrReplaceTempView('tempview1')
df1.createOrReplaceTempView('tempview2')

In [55]:
#1.show item detailes supplied by mr x
pd=spark.sql('select * from tempview1 as t join tempview2 as t1 on t.sup_id==t1.sup_id where sup_name=="X" ')
pd.show()

+------+--------+--------+------+------+--------+
|itemid|itemname|itemcost|sup_id|sup_id|Sup_name|
+------+--------+--------+------+------+--------+
|    15|   chalk|    1000|     1|     1|       X|
|    11|  Pencil|     900|     1|     1|       X|
+------+--------+--------+------+------+--------+



In [66]:
#2.show supplier details who has supplied items with cost>1000
pd=spark.sql('select * from tempview2 as t join tempview1 as t1 on t.sup_id==t1.sup_id where itemcost>1000')
pd.show()

+------+--------+------+--------+--------+------+
|sup_id|Sup_name|itemid|itemname|itemcost|sup_id|
+------+--------+------+--------+--------+------+
|     2|    null|    13|     Pen|    1500|     2|
|     3|       Z|    15|  Duster|    1600|     3|
+------+--------+------+--------+--------+------+



In [146]:
#3.show all item details whos supplier details are not available
pd1=spark.sql('select * from tempview1 as t join tempview2 as t1 on t.sup_id==t1.sup_id where t1.sup_name is null')
pd1.show()

+------+--------+--------+------+------+--------+
|itemid|itemname|itemcost|sup_id|sup_id|sup_name|
+------+--------+--------+------+------+--------+
|    13|     Pen|    1500|     2|     2|    null|
|    78|   Scale|     100|     2|     2|    null|
+------+--------+--------+------+------+--------+



In [106]:
#4.show item details whose supplier details are available
pd1=spark.sql('select * from tempview1 as t join tempview2 as t1 on t.sup_id==t1.sup_id where sup_name!=" "')
pd1.show()

+------+--------+--------+------+------+--------+
|itemid|itemname|itemcost|sup_id|sup_id|sup_name|
+------+--------+--------+------+------+--------+
|    15|   chalk|    1000|     1|     1|       X|
|    11|  Pencil|     900|     1|     1|       X|
|    15|  Duster|    1600|     3|     3|       Z|
|    89|   chalk|     600|     4|     4|       A|
+------+--------+--------+------+------+--------+



In [142]:
#5.show item details along with supplier details for such items,for which supplier details are  available and item name starts with 'b'
pd3=spark.sql('select * from tempview1 as t join tempview2 as t1 on t.sup_id==t1.sup_id where t.itemname like "c%"')
pd3.show()

+------+--------+--------+------+------+--------+
|itemid|itemname|itemcost|sup_id|sup_id|sup_name|
+------+--------+--------+------+------+--------+
|    15|   chalk|    1000|     1|     1|       X|
|    89|   chalk|     600|     4|     4|       A|
+------+--------+--------+------+------+--------+



In [115]:
#6.Show supplier wise number of items supplied, sum ,min ,max total of itemcost .
pd4=spark.sql('select t1.sup_id,max(t.itemcost),min(t.itemcost),sum(t.itemcost) from tempview1 as t join tempview2 as t1 on t.sup_id==t1.sup_id group by(t1.sup_id) ')
pd4.show()

+------+-------------+-------------+-------------+
|sup_id|max(itemcost)|min(itemcost)|sum(itemcost)|
+------+-------------+-------------+-------------+
|     1|         1000|          900|         1900|
|     3|         1600|         1600|         1600|
|     4|          600|          600|          600|
|     2|         1500|          100|         1600|
+------+-------------+-------------+-------------+



In [129]:
#7.join overall items available in 2 stores
pd6=df.join(df1,df.sup_id==df1.sup_id)
pd6.show()

+------+--------+--------+------+------+--------+
|itemid|itemname|itemcost|sup_id|sup_id|sup_name|
+------+--------+--------+------+------+--------+
|    15|   chalk|    1000|     1|     1|       X|
|    11|  Pencil|     900|     1|     1|       X|
|    13|     Pen|    1500|     2|     2|    null|
|    15|  Duster|    1600|     3|     3|       Z|
|    78|   Scale|     100|     2|     2|    null|
|    89|   chalk|     600|     4|     4|       A|
+------+--------+--------+------+------+--------+

